<a href="https://colab.research.google.com/github/SJinLee/DS/blob/main/01_crawling_naver_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 워드 클라우드 그려보기

목차

1. colab notebook
2. 설치
3. 네이버 뉴스 크롤링
4. pandas, DataFrame
5. 워드클라우드
6. 형태소 분석
7. 명사들의 워드클라우드
8. 분야별 워드클라우드

### 1. colab notebook

* 셀(cell)들로 이루어져 있다. 코드셀과 텍스트셀이 있다.
* markdown 형식으로 문서작성을 할 수 있다.
* python 프로그램을 실행할 수 있다.
* 셀에 입력된 프로그램을 실행하려면 Shift+Enter 를 하면 된다.

### 2. 설치

* KoNLPy

In [ ]:
!python3 -m pip install konlpy 

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

* 나눔폰트: https://hangeul.naver.com/2017/nanum

In [ ]:
!apt install fonts-nanum

### 3. 네이버 뉴스 크롤링

In [27]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError
import pandas as pd

def read_url(url):
    headers = {'User-Agent':' Mozilla/5.0 (Windows NT 6.1; WOW64; rv:12.0) Gecko/20100101 Firefox/12.0'}
    req = Request(url=url,headers=headers)
    try:
        handler = urlopen(req)
    except HTTPError as e:
        content = e.read()
    return handler.read()

In [28]:
base_url = "https://news.naver.com"

[네이버 뉴스 소스 보기](https://drive.google.com/file/d/1y2JNxaSDeeUpHDJveg1Y3BDEgSGtPnRh/view?usp=sharing) : 네이버 뉴스 화면에서 F12를 누른다.

In [29]:
data = read_url(base_url)

In [ ]:
data.decode(encoding='euc-kr')

In [31]:
soup = BeautifulSoup(data, "html.parser")
total_data = soup.find_all(attrs={'class': 'main_component droppable'})

In [32]:
category = [x.find(attrs={'class': 'tit_sec'}) for x in total_data]
category = [x.find('a').string if x is not None else '' for x in category]

In [33]:
data_str = []
for cat,data in zip(category[1:],total_data[1:]):
    news_list2 = data.find_all('li')
    news_list = [x.find('a') for x in news_list2]
    titles = [x.find('strong').text for x in news_list]
    news_url = [x.attrs['href'] for x in news_list]
    soups = [BeautifulSoup(read_url(x), "html.parser") for x in news_url]
    article_bodies = [x.find(attrs={'id': 'articleBodyContents'}) for x in soups]
    for ab in article_bodies:
      ab.find('script').decompose()
    for i in range(len(news_list)):
        data_str.append([cat,titles[i],news_url[i],str(article_bodies[i].text)])


### 4. pandas, DataFrame

In [34]:
columns = ['category','title','news_url','article_body']
df = pd.DataFrame(data_str,range(len(data_str)),columns)

In [ ]:
df

### 5. 워드클라우드(wordcloud)

In [35]:
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

In [36]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [37]:
text = ' '.join(df['article_body'])

### 6. 형태소 분석

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
kkma.nouns(u'질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.')

In [39]:
noun = kkma.nouns(text)

### 7. 명사들의 워드클라우드

In [40]:
wordcloud = WordCloud(font_path=font_path,background_color='white').generate(' '.join(noun))

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(wordcloud, interpolation='lanczos')
plt.axis('off')
plt.show()

### 8. 분야별 워드클라우드


In [ ]:
df.columns

In [ ]:
df['category'].unique()

* 정치

In [22]:
index = df['category']=='정치'
df1 = df.loc[index]
text = ' '.join(df1['article_body'])

In [23]:
noun = kkma.nouns(text)

In [24]:
wordcloud = WordCloud(font_path=font_path, background_color='white').generate(' '.join(noun))

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(wordcloud, interpolation='lanczos')
plt.axis('off')
plt.show() 

* 경제

**연습문제**

* 사회, 생활/문화, 세계, IT/과학 분야의 워드클라우드를 그려보자.

### 코드모음

5. 워드클라우드

<code>
text = ' '.join(df['article_body'])
wordcloud = WordCloud(font_path=font_path,background_color='white').generate(text)
plt.figure(figsize=(10,8))
plt.imshow(wordcloud, interpolation='lanczos')
plt.axis('off')
plt.show() 
</code>